In [12]:
# 데이터 처리 라이브러리
import os
import os.path as pth
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

# Tensorflow 관련 라이브러리
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, BatchNormalization, PReLU
from tensorflow.keras.layers import Flatten, Activation, Dense, GlobalAveragePooling2D, Softmax
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# tensorflow에서 사용할 수 없는 기능을 구현하는 기여 저장소
from tensorflow_addons import optimizers


# GPU 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        
        tf.config.experimental.set_memory_growth(gpus[0], True)
        
#         tf.config.experimental.set_virtual_device_configuration(
#             gpus[0],
#             [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=15000)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

In [4]:
tf.__version__

'2.3.0'

In [18]:
train_tfrecord_path = pth.join('./data/emotion_crop_re_tf', 'tf_record_train_7.tfrecords')
valid_tfrecord_path = pth.join('./data/emotion_crop_re_tf', 'tf_record_valid_7.tfrecords')

# BUFFER_SIZE, BATCH_SIZE
BUFFER_SIZE = 256
BATCH_SIZE = 90
NUM_CLASS = 7



image_feature_description = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'img_id': tf.io.FixedLenFeature([], tf.string),
}



def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)

def map_func(target_record):
    img = target_record['image_raw']
    label = int(target_record['img_id'])
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.dtypes.cast(img, tf.float32)
    return img, label

def prep_func(image, label):
    print(type(label))

    result_image = image / 255
    result_image = tf.image.resize(result_image, (224,224))

    onehot_label = tf.one_hot(label, depth=NUM_CLASS)
    return result_image, onehot_label

    

dataset = tf.data.TFRecordDataset(train_tfrecord_path, compression_type='GZIP')
dataset = dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

valid_dataset = tf.data.TFRecordDataset(valid_tfrecord_path, compression_type='GZIP')
valid_dataset = valid_dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.shuffle(BUFFER_SIZE)
valid_dataset = valid_dataset.batch(BATCH_SIZE)
valid_dataset = valid_dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


In [19]:
earlystop = EarlyStopping(patience = 5)

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                            patience = 2,
                                            factor = 0.5,
                                            min_lr = 1e-7,
                                            verbose = 1)

model_check = ModelCheckpoint(filepath = './model/emo7_facial_mobile_224_224_PReLU.h5',
                              monitor = 'val_loss',
                              save_best_only = True)

callbacks = [earlystop, learning_rate_reduction, model_check]

In [20]:
from tensorflow.keras.applications import MobileNetV2
mobilenet = MobileNetV2(weights = 'imagenet', include_top = False, input_shape = (224,224,3))


for layer in mobilenet.layers:
    layer.trainable = True
    
for (i, layer) in enumerate(mobilenet.layers):
    print(str(i), layer.__class__.__name__, layer.trainable)
    
    
model = Sequential()
model.add(mobilenet)
model.add(GlobalAveragePooling2D())
model.add(Dense(7))
model.add(BatchNormalization())
model.add(PReLU())
model.add(Softmax(dtype='float32', name='softmax'))
model.summary()

0 InputLayer True
1 ZeroPadding2D True
2 Conv2D True
3 BatchNormalization True
4 ReLU True
5 DepthwiseConv2D True
6 BatchNormalization True
7 ReLU True
8 Conv2D True
9 BatchNormalization True
10 Conv2D True
11 BatchNormalization True
12 ReLU True
13 ZeroPadding2D True
14 DepthwiseConv2D True
15 BatchNormalization True
16 ReLU True
17 Conv2D True
18 BatchNormalization True
19 Conv2D True
20 BatchNormalization True
21 ReLU True
22 DepthwiseConv2D True
23 BatchNormalization True
24 ReLU True
25 Conv2D True
26 BatchNormalization True
27 Add True
28 Conv2D True
29 BatchNormalization True
30 ReLU True
31 ZeroPadding2D True
32 DepthwiseConv2D True
33 BatchNormalization True
34 ReLU True
35 Conv2D True
36 BatchNormalization True
37 Conv2D True
38 BatchNormalization True
39 ReLU True
40 DepthwiseConv2D True
41 BatchNormalization True
42 ReLU True
43 Conv2D True
44 BatchNormalization True
45 Add True
46 Conv2D True
47 BatchNormalization True
48 ReLU True
49 DepthwiseConv2D True
50 BatchNormaliza

In [4]:
## 시간이 너무 오래 걸림

from tensorflow.keras.applications import EfficientNetB5
efnet = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(270,270,3))

for layer in efnet.layers:
    layer.trainable = True
    
for (i, layer) in enumerate(efnet.layers):
    print(str(i), layer.__class__.__name__, layer.trainable)
    
    
model = Sequential()
model.add(efnet)
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dense(8))
model.add(PReLU())
model.add(Softmax(dtype='float32', name='softmax'))
model.summary()

0 InputLayer True
1 Rescaling True
2 Normalization True
3 ZeroPadding2D True
4 Conv2D True
5 BatchNormalization True
6 Activation True
7 DepthwiseConv2D True
8 BatchNormalization True
9 Activation True
10 GlobalAveragePooling2D True
11 Reshape True
12 Conv2D True
13 Conv2D True
14 Multiply True
15 Conv2D True
16 BatchNormalization True
17 DepthwiseConv2D True
18 BatchNormalization True
19 Activation True
20 GlobalAveragePooling2D True
21 Reshape True
22 Conv2D True
23 Conv2D True
24 Multiply True
25 Conv2D True
26 BatchNormalization True
27 Dropout True
28 Add True
29 DepthwiseConv2D True
30 BatchNormalization True
31 Activation True
32 GlobalAveragePooling2D True
33 Reshape True
34 Conv2D True
35 Conv2D True
36 Multiply True
37 Conv2D True
38 BatchNormalization True
39 Dropout True
40 Add True
41 Conv2D True
42 BatchNormalization True
43 Activation True
44 ZeroPadding2D True
45 DepthwiseConv2D True
46 BatchNormalization True
47 Activation True
48 GlobalAveragePooling2D True
49 Reshape

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb5 (Functional)  (None, 9, 9, 2048)        28513527  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 2048)              8192      
_________________________________________________________________
dense (Dense)                (None, 8)                 16392     
_________________________________________________________________
p_re_lu (PReLU)              (None, 8)                 8         
_________________________________________________________________
softmax (Softmax)            (None, 8)                 0         
Total params: 28,538,119
Trainable params: 28,361,280
Non-trainable params: 176,839
______________________________________

In [9]:
## 실패 ## 정확도 10퍼센트대

model = Sequential()

model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(3,3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(BatchNormalization())


model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(5))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 72, 72, 96)        34944     
_________________________________________________________________
activation_5 (Activation)    (None, 72, 72, 96)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 96)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 36, 36, 96)        384       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 256)       2973952   
_________________________________________________________________
activation_6 (Activation)    (None, 26, 26, 256)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 256)      

In [21]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RectifiedAdam(learning_rate=0.001),
              metrics=['accuracy'])

In [22]:
history = model.fit(dataset,
                    epochs=100,
                    validation_data=valid_dataset,
                    callbacks=callbacks,
                   )

Epoch 1/100
      2/Unknown - 1s 258ms/step - loss: 1.8630 - accuracy: 0.1667WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1957s vs `on_train_batch_end` time: 0.3195s). Check your callbacks.
606/606 [==============================] - 365s 602ms/step - loss: 0.5727 - accuracy: 0.8210 - val_loss: 1.0569 - val_accuracy: 0.5454
Epoch 2/100
606/606 [==============================] - 368s 607ms/step - loss: 0.2564 - accuracy: 0.8731 - val_loss: 0.7687 - val_accuracy: 0.6516
Epoch 3/100
606/606 [==============================] - 368s 608ms/step - loss: 0.1982 - accuracy: 0.8592 - val_loss: 1.7510 - val_accuracy: 0.2072
Epoch 4/100
606/606 [==============================] - ETA: 0s - loss: 0.1905 - accuracy: 0.8551
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
606/606 [==============================] - 368s 608ms/step - loss: 0.1905 - accuracy: 0.8551 - val_loss: 2.0098 - val_accuracy: 0.1312
Epoch 5